In [8]:
import numpy as np
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve

def cheb_pts(N):
    k = np.arange(N)
    x = np.cos(np.pi * k / (N - 1))
    return x

def compute_D2(N):
    x = cheb_pts(N)
    D2 = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            if i != j:
                D2[i, j] = ((-1) ** (i + j)) / (x[i] - x[j])
    D2 = D2 - np.diag(np.sum(D2, axis=1))
    return D2

def residual(u, f):
    N = len(u)
    x = cheb_pts(N)
    D2 = compute_D2(N)
    LHS = np.dot(D2, u) + u ** 5
    RHS = f(x)
    return LHS - RHS

def solve_bvp(N, f, tol=1e-8, max_iter=1000):
    x = cheb_pts(N)
    u = np.zeros(N)
    iteration = 0
    while True:
        residue = residual(u, f)
        D2 = compute_D2(N)
        J = D2 + np.diag(5 * u ** 4)
        
        # Construct the banded matrix J using scipy.sparse.diags
        diagonals = [J.diagonal(), np.diag(J, k=1), np.diag(J, k=-1)]
        ab = diags(diagonals, [0, 1, -1])
        
        delta_u = spsolve(ab, -residue)
        u += delta_u
        iteration += 1
        if np.linalg.norm(delta_u, np.inf) < tol or iteration >= max_iter:
            break
    return u, x

# Example of usage:
def f_example(x):
    return np.sin(np.pi * x)

N = 100  # Number of grid points
u, x = solve_bvp(N, f_example)

# Display or use the result as needed
print("Computed solution u at Chebyshev grid points:")
print(u)


C:\Users\srish\anaconda3\Lib\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:214: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',


Computed solution u at Chebyshev grid points:
[ 4.78172245e+01  8.89355969e+01 -3.11634084e+01  7.46993568e+00
 -1.09836918e+01  8.44287429e+00 -6.55019674e+00  5.62704162e+00
 -5.07342855e+00  4.84219311e+00 -4.68338240e+00  4.78390377e+00
  1.29286662e+01 -2.38814666e+01 -8.19399752e+01 -2.61241420e+02
 -6.47136907e+01  5.36694682e+00 -5.10934673e+00  5.14727407e+00
  6.79865162e+01 -7.34304585e+00 -3.87482762e+01 -1.65736993e+01
 -2.63192318e+00  4.72983954e+02 -1.33579144e+00  1.35215253e+01
 -6.58659812e+00  6.32443871e+00 -6.15148450e+00  6.04194267e+00
 -5.98943216e+00  5.91375216e+00 -5.87787917e+00  5.86146072e+00
 -5.85134515e+00  5.85656659e+00 -5.86521364e+00  5.88666792e+00
 -5.91037973e+00  5.94544893e+00 -5.98239648e+00  6.02969068e+00
 -6.07857848e+00  6.13622498e+00 -6.19298535e+00  6.25016131e+00
 -6.32168711e+00  3.87782884e+01 -6.75375626e+00  6.82577261e+00
 -6.93931087e+00  7.08821679e+00 -7.25954348e+00  7.46266590e+00
 -7.69642916e+00  7.97577075e+00 -4.85256789

In [9]:
import sympy as sp

# Define the symbolic variable
x = sp.symbols('x')

# Define the function u(x)
u = sp.exp(x) * (x**2 - 1)

# Compute the derivatives
u_xx = sp.diff(u, x, x)
u_5 = u**5

# Calculate f(x) = u_xx + u^5
f_x = u_xx + u_5

# Print the expression for f(x)
print("The source term f(x) that satisfies the differential equation is:")
print(f_x)


The source term f(x) that satisfies the differential equation is:
(x**2 - 1)**5*exp(5*x) + (x**2 + 4*x + 1)*exp(x)
